In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

from modules.data_fetcher import download_historical_data


In [ ]:
symbol= 'BTC-USDT'

df_BTC = download_historical_data(symbol,'1hour').iloc[-3000:]
df_BTC["Return"] = df_BTC.Close.pct_change()
df_BTC.dropna(inplace=True)
print(df_BTC.shape)
df_BTC.head()

train = df_BTC[:df_BTC.shape[0]//2]
test = df_BTC[df_BTC.shape[0]//2:]
df_BTC.head()

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.forecasting.theta import ThetaModel


def rolling_forecast(df: pd.DataFrame, train_len: int, horizon: int, window: int, method: str) -> list:
    
    total_len = train_len + horizon
    end_idx = train_len

    if method == 'last_season':
        pred_last_season = []
        
        for i in range(train_len, total_len, window):
            last_season = df['Close'][:i].iloc[-window:].values
            pred_last_season.extend(last_season)
            
        return pred_last_season
    
    elif method == 'theta':
        pred_theta = []
        
        for i in range(train_len, total_len, window):
            tm = ThetaModel(endog=df['Close'][:i], period=52)
            res = tm.fit()
            predictions = res.forecast(window)
            pred_theta.extend(predictions)
            
        #print(res.summary())

        return pred_theta
              
    elif method == 'tes':
        pred_tes = []
        
        for i in range(train_len, total_len, window):
            tes = ExponentialSmoothing(
                df['Close'][:i],
                trend='add',
                seasonal='add',
                seasonal_periods=52,
                initialization_method='estimated'
            ).fit()
            
            predictions = tes.forecast(window)
            pred_tes.extend(predictions)

    return pred_tes

In [ ]:
TRAIN_LEN = len(train)
HORIZON = len(test)
WINDOW = 50

pred_last_season = rolling_forecast(df, TRAIN_LEN, HORIZON, WINDOW, 'last_season')[:len(test)]
pred_theta = rolling_forecast(df, TRAIN_LEN, HORIZON, WINDOW, 'theta')[:len(test)]
pred_tes = rolling_forecast(df, TRAIN_LEN, HORIZON, WINDOW, 'tes')[:len(test)]

test = test.copy()

test['pred_last_season'] = pred_last_season# [:-(len(pred_last_season)-len(test))]
test['pred_theta'] = pred_theta# [:-(len(pred_theta)-len(test))]
test['pred_tes'] = pred_tes# [:-(len(pred_tes)-len(test))]

test.head()

In [ ]:
fig, ax = plt.subplots(figsize=(25,8))

ax.plot(df_BTC['Close'])
ax.plot(test['Close'], 'b-', label='actual')
ax.plot(test['pred_last_season'], 'r:', label='baseline')
ax.plot(test['pred_theta'], 'g-.', label='Theta')
ax.plot(test['pred_tes'], 'k--', label='TES')

ax.set_xlabel('Time')
ax.set_ylabel('Price')

ax.legend(loc='best')